### Deliverable 3: Optimize the Model

#### Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"],1)
application_df.head()

C:\Users\carol\.conda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
appl_type_count = application_df.APPLICATION_TYPE.value_counts()

#### Removing noise variables from string columns

In [5]:
# Determine which values to replace if counts are less than 750?
replace_application = list(appl_type_count[appl_type_count < 750].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()

In [7]:
# Determine which values to replace if counts are less than 2000?
replace_class = list(classification_count[classification_count < 2000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\carol\.conda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

C:\Users\carol\.conda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### First Attempt: Compile, Train and Evaluate the Model

#### Adding hidden layers

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3120      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,511
Trainable params: 6,511
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [16]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5772 - accuracy: 0.7188
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5616 - accuracy: 0.7281
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5592 - accuracy: 0.7283
Epoch 4/100
804/804 [==============================] - 1s 943us/step - loss: 0.5574 - accuracy: 0.7292
Epoch 5/100
778/804 [============================>.] - ETA: 0s - loss: 0.5566 - accuracy: 0.7300
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7301
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5553 - accuracy: 0.7299
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7333
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5546 - accuracy: 0.7315
Epoch 9/100
804/804 [==============================]

804/804 [==============================] - 1s 933us/step - loss: 0.5440 - accuracy: 0.7372
Epoch 65/100
754/804 [===========================>..] - ETA: 0s - loss: 0.5451 - accuracy: 0.7367
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 883us/step - loss: 0.5437 - accuracy: 0.7381
Epoch 66/100
804/804 [==============================] - 1s 874us/step - loss: 0.5432 - accuracy: 0.7377
Epoch 67/100
804/804 [==============================] - 1s 832us/step - loss: 0.5437 - accuracy: 0.7379
Epoch 68/100
804/804 [==============================] - 1s 964us/step - loss: 0.5435 - accuracy: 0.7382
Epoch 69/100
804/804 [==============================] - 1s 851us/step - loss: 0.5433 - accuracy: 0.7377
Epoch 70/100
746/804 [==========================>...] - ETA: 0s - loss: 0.5433 - accuracy: 0.7379
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 827us/step - loss: 0.5432 - accuracy: 0.7383
Epoch 7

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6090 - accuracy: 0.7207 - 278ms/epoch - 1ms/step
Loss: 0.6089831590652466, Accuracy: 0.7206997275352478


### Second Attempt: Compile, Train and Evaluate the Model

#### Increasing neurons on the hidden layers

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 45
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Thrid hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               3900      
                                                                 
 dense_5 (Dense)             (None, 45)                4545      
                                                                 
 dense_6 (Dense)             (None, 20)                920       
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 9,386
Trainable params: 9,386
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5734 - accuracy: 0.7197
Epoch 2/100
804/804 [==============================] - 1s 874us/step - loss: 0.5603 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 1s 855us/step - loss: 0.5581 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 1s 980us/step - loss: 0.5566 - accuracy: 0.7305
Epoch 5/100
744/804 [==========================>...] - ETA: 0s - loss: 0.5539 - accuracy: 0.7328
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 832us/step - loss: 0.5552 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 1s 886us/step - loss: 0.5541 - accuracy: 0.7316
Epoch 7/100
804/804 [==============================] - 1s 797us/step - loss: 0.5540 - accuracy: 0.7330
Epoch 8/100
804/804 [==============================] - 1s 820us/step - loss: 0.5535 - accuracy: 0.7336
Epoch 9/100
804/804 [===================

804/804 [==============================] - 1s 875us/step - loss: 0.5435 - accuracy: 0.7378
Epoch 65/100
803/804 [============================>.] - ETA: 0s - loss: 0.5429 - accuracy: 0.7385
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5430 - accuracy: 0.7384
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5426 - accuracy: 0.7389
Epoch 67/100
804/804 [==============================] - 1s 837us/step - loss: 0.5427 - accuracy: 0.7388
Epoch 68/100
804/804 [==============================] - 1s 871us/step - loss: 0.5430 - accuracy: 0.7377
Epoch 69/100
804/804 [==============================] - 1s 874us/step - loss: 0.5427 - accuracy: 0.7387
Epoch 70/100
802/804 [============================>.] - ETA: 0s - loss: 0.5423 - accuracy: 0.7385
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 845us/step - loss: 0.5424 - accuracy: 0.7385
Epoch 71/10

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6522 - accuracy: 0.7226 - 243ms/epoch - 906us/step
Loss: 0.652212917804718, Accuracy: 0.7225655913352966


### Third Attempt: Compile, Train and Evaluate the Model

#### Changing activation function on hidden layers

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 45
hidden_nodes_layer3 =20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Thrid hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                3120      
                                                                 
 dense_9 (Dense)             (None, 45)                3645      
                                                                 
 dense_10 (Dense)            (None, 20)                920       
                                                                 
 dense_11 (Dense)            (None, 1)                 21        
                                                                 
Total params: 7,706
Trainable params: 7,706
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#### Adding more epochs

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5723 - accuracy: 0.7196
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5610 - accuracy: 0.7279
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5591 - accuracy: 0.7278
Epoch 4/200
804/804 [==============================] - 1s 860us/step - loss: 0.5577 - accuracy: 0.7286
Epoch 5/200
751/804 [===========================>..] - ETA: 0s - loss: 0.5585 - accuracy: 0.7290
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 822us/step - loss: 0.5572 - accuracy: 0.7296
Epoch 6/200
804/804 [==============================] - 1s 785us/step - loss: 0.5560 - accuracy: 0.7285
Epoch 7/200
804/804 [==============================] - 1s 799us/step - loss: 0.5552 - accuracy: 0.7298
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5543 - accuracy: 0.7303
Epoch 9/200
804/804 [=========================

804/804 [==============================] - 1s 1ms/step - loss: 0.5423 - accuracy: 0.7384
Epoch 65/200
799/804 [============================>.] - ETA: 0s - loss: 0.5428 - accuracy: 0.7376
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5427 - accuracy: 0.7377
Epoch 66/200
804/804 [==============================] - 1s 791us/step - loss: 0.5423 - accuracy: 0.7382
Epoch 67/200
804/804 [==============================] - 1s 889us/step - loss: 0.5423 - accuracy: 0.7389
Epoch 68/200
804/804 [==============================] - 1s 945us/step - loss: 0.5424 - accuracy: 0.7381
Epoch 69/200
804/804 [==============================] - 1s 896us/step - loss: 0.5422 - accuracy: 0.7381
Epoch 70/200
760/804 [===========================>..] - ETA: 0s - loss: 0.5427 - accuracy: 0.7385
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5423 - accuracy: 0.7386
Epoch 71/200


Epoch 126/200
804/804 [==============================] - 1s 814us/step - loss: 0.5408 - accuracy: 0.7393
Epoch 127/200
804/804 [==============================] - 1s 776us/step - loss: 0.5407 - accuracy: 0.7391
Epoch 128/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5405 - accuracy: 0.7397
Epoch 129/200
804/804 [==============================] - 1s 984us/step - loss: 0.5402 - accuracy: 0.7397
Epoch 130/200
792/804 [============================>.] - ETA: 0s - loss: 0.5411 - accuracy: 0.7384
Epoch 130: saving model to checkpoints\weights.130.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5406 - accuracy: 0.7388
Epoch 131/200
804/804 [==============================] - 1s 971us/step - loss: 0.5399 - accuracy: 0.7398
Epoch 132/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5403 - accuracy: 0.7390
Epoch 133/200
804/804 [==============================] - 1s 796us/step - loss: 0.5400 - accuracy: 0.7395
Epoch 134/200
804/804 [==

804/804 [==============================] - 1s 801us/step - loss: 0.5398 - accuracy: 0.7397
Epoch 189/200
804/804 [==============================] - 1s 857us/step - loss: 0.5393 - accuracy: 0.7396
Epoch 190/200
798/804 [============================>.] - ETA: 0s - loss: 0.5406 - accuracy: 0.7388
Epoch 190: saving model to checkpoints\weights.190.hdf5
804/804 [==============================] - 1s 980us/step - loss: 0.5404 - accuracy: 0.7390
Epoch 191/200
804/804 [==============================] - 1s 946us/step - loss: 0.5410 - accuracy: 0.7392
Epoch 192/200
804/804 [==============================] - 1s 838us/step - loss: 0.5401 - accuracy: 0.7389
Epoch 193/200
804/804 [==============================] - 1s 779us/step - loss: 0.5398 - accuracy: 0.7399
Epoch 194/200
804/804 [==============================] - 1s 934us/step - loss: 0.5420 - accuracy: 0.7395
Epoch 195/200
765/804 [===========================>..] - ETA: 0s - loss: 0.5424 - accuracy: 0.7363
Epoch 195: saving model to checkpoints\

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5783 - accuracy: 0.7193 - 278ms/epoch - 1ms/step
Loss: 0.5782661437988281, Accuracy: 0.7193002700805664


In [28]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")